In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import os #deal with directories and can work on all OS

In [ ]:
## Path definitions
PROJECT_ROOT = os.path.join(os.getcwd(), '..') #Main file (in this case "AI_Analysis_and_Chatbot")
EXCEL_FILE_PATH = os.path.join(PROJECT_ROOT, 'Data', 'amazon_product_reviews.xlsx') #gets path with file name and folder

In [15]:
## Data cleaning functions

# Cleaning numbers
def clean_num (series): #remove symbols from numbers and and convert to float
    if series.dtype == 'object': #when reading it appears as text (so we dont take numbers without symbols)
        cleaned_series = (series.astype(str) #cast to string for consistency
                      .str.replace('â‚¹', '', regex=False) #delete symbol (replace with empty string)
                      .str.replace(',', '', regex=False) #so they can be dealt with as floats/ints not strings
                      .str.replace('%', '', regex=False)
                      .str.strip()) #remove spaces before/after string
        return pd.to_numeric(cleaned_series, errors='coerce')
    return series

# Cleaning text
def clean_txt(series):
    if series.dtype == 'object':
        return (series.astype(str)
                .str.replace(r'[ã€\s?\[?\u200b\ufeff\u00a0\u201d\u201c\u2013\u2022âœ“â€¢â˜…ã€‘]', '', regex=True) #remove symbols
                .str.strip()
                .str.replace(r'\s+', ' ', regex=True) #replace multispace with one space
                )
    return series

In [16]:
## Data Reading

try:
    df = pd.read_excel(EXCEL_FILE_PATH, engine='openpyxl') #openpyxl is modern engine for .xlsx format
    print(f"Successfully loaded {len(df)} records.") #prove file was read correctly and how many records we have
except FileNotFoundError:
    print(f"ERROR: File not in {EXCEL_FILE_PATH}")
    exit()


Successfully loaded 1465 records.


In [23]:
## Data Cleaning

# Divide columns so we can clean them as needed
Num_Cols = ['discounted_price', 'actual_price', 'discount_percentage', 'rating', 'rating_count']
Txt_Cols = ['product_name','about_product', 'user_name','review_title','review_content'] 

# Clean numbers
for col in Num_Cols:
    if col in df.columns:
        df[col] = clean_num(df[col])
        
# Clean text
for col in Txt_Cols:
    if col in df.columns:
        df[col] = clean_txt(df[col])

print("Price, count, and percentage columns cleaned and converted to numeric.")
print(df[Num_Cols].head(2)) #test make sure it is correct


Price, count, and percentage columns cleaned and converted to numeric.
   discounted_price  actual_price  discount_percentage  rating  rating_count
0             399.0        1099.0                 0.64     4.2       24269.0
1             399.0        1099.0                 0.64     4.2       24269.0


In [40]:
## Explode (seperate each user to new row)
Cols_to_Split = ['user_id', 'user_name', 'review_id', 'review_title', 'review_content'] #cols have multiple ',' separated values
for col in Cols_to_Split:
    if col in df.columns:
        df[col] = df[col].astype(str)#.str.split(',')
        df[col] = df[col].str.replace(r'[\s\W_]+', ' ', regex=True) #clean
        df[col] = df[col].str.split(',') #split on comma

Org_Rows = len(df) #duplicate rows (Org = orginal)
df = df.explode('user_id', ignore_index=True) #pandas explode method
df = df.reset_index(drop=True) #reset to prevent errors
Exp_Rows = len(df) # exploded rows (Exp = exploded)

print(f"Original rows: {Org_Rows}")
print(f"Rows after separating: {Exp_Rows}")

for col in Cols_to_Split: #explode rest of columns
    if col != 'user_id' and col in df.columns:
        Exp_Data = df[col].explode().to_list() 
        df[col] = pd.Series(Exp_Data) 
        Cleaned_Data = (df[col].astype(str) #cleaning
                   .str.replace(r"['\"\[\]\\]", '', regex=True) #remove quotes, brackets, and slashes
                   .str.strip()) 
        if col in ['user_name', 'review_title', 'review_content']:
        #    Cleaned_Data = Cleaned_Data.str.replace(r'[^\w\s]', ' ', regex=True)
            Cleaned_Data = Cleaned_Data.str.replace(r'([a-z])([A-Z])', r'\1 \2', regex=True) #insert space based on capitalization
            Cleaned_Data = Cleaned_Data.str.replace(r'\s+', ' ', regex=True).str.strip()
        df[col] = Cleaned_Data
    
df['user_id'] = df['user_id'].astype(str).str.strip()
display(df[['product_id','user_name','review_title']].head(3))

Original rows: 11503
Rows after separating: 11503


,product_id,user_name,review_title
0,B07JW9H4J1,Manav,Satisfied
1,B07JW9H4J1,Adarshgupta,Chargingisreallyfast
2,B07JW9H4J1,Sundeep,Valueformoney


In [52]:
## Data Analysis
Brand_Colors = ['#4f008c','#c2a6cf','#e4d9eb','#e4e9ee']

# Top 5 rated products analysis
if 'product_id' in df.columns and 'rating' in df.columns:

    avg_rate = df.groupby('product_id')['rating'].mean().sort_values(ascending=False) #avg and sorts from high to low
    Top_5_Rate = avg_rate.head(5)
    print ("Top 5 rated products")
    display(Top_5_Rate.to_frame(name='Average Rating').T) # .T is transposition, makes table easier to read

    #Top 5 visualization
    fig_top_5 = px.bar(Top_5_Rate,
                       x = Top_5_Rate.index,
                       y = 'rating',
                       color = Top_5_Rate.index,
                       color_discrete_sequence= Brand_Colors,
                       title = "Top 5 rated products",
                       labels={'x': 'Product ID', 'rating':'Average Rating'})
    fig_top_5.show()

Top 5 rated products


product_id,B0BQRJ3C47,B09ZHCJDP1,B0BP7XLX48,B0B53DS4TF,B0BR4F878Q
Average Rating,5.0,5.0,5.0,4.8,4.8


In [63]:
# Breakdown by category analysis
if 'category' in df.columns and 'rating' in df.columns:
    Avg_Cat_Rate = df.groupby('category')['rating'].mean().sort_values(ascending=False).round(2) #average category rating
    print("\nAverage rating breakdown by category")
    display(Avg_Cat_Rate.head(10).to_frame(name='Average Rating'))

    #category analysis visualization
    fig_cat_rate = px.bar(Avg_Cat_Rate.head(10),
        x = Avg_Cat_Rate.head(10).index,
        y = 'rating',
        title = "Categories sorted by average rating (Top 10)",
        labels={'x': 'Category', 'rating': 'Average Rating'},
        color = Avg_Cat_Rate.head(10).index,
        color_discrete_sequence= Brand_Colors)
    fig_cat_rate.show()


Average rating breakdown by category


,Average Rating
category,
Computers&Accessories|Tablets,4.6
Computers&Accessories|NetworkingDevices|NetworkAdapters|PowerLANAdapters,4.5
Electronics|Cameras&Photography|Accessories|Film,4.5
Computers&Accessories|Components|Memory,4.5
Electronics|HomeAudio|MediaStreamingDevices|StreamingClients,4.5
OfficeProducts|OfficeElectronics|Calculators|Basic,4.5
HomeImprovement|Electrical|CordManagement,4.5
"Home&Kitchen|Kitchen&HomeAppliances|Coffee,Tea&Espresso|CoffeePresses",4.5
Home&Kitchen|Kitchen&HomeAppliances|SmallKitchenAppliances|SmallApplianceParts&Accessories,4.5


In [ ]:
## Feature engineering
New_Col = 'Customer_Feedback'

if 'review_title' in df.columns and 'review_content' in df.columns:
    df[New_Col] = df['review_title'].astype(str) + " " + df['review_content'].astype(str)
    print(f"\nFeature Engineering: Successfully created '{New_Col}' for sentiment analysis.")
else:
    print(f"ERROR: {New_Col} could not be created")

os.makedirs('Data',exist_ok=True) #create Data directory if it doesn't exist
df.to_pickle(os.path.join('Data','cleaned_sales_data.pkl'))
print("Data saved as 'cleaned_sales_data.pkl'")



Feature Engineering: Successfully created 'Customer_Feedback' for sentiment analysis.
